Write a function to find the longest common prefix string amongst an array of strings.

If there is no common prefix, return an empty string "".

Example 1:
```
Input: ["flower","flow","flight"]
Output: "fl"
```
Example 2:
```
Input: ["dog","racecar","car"]
Output: ""
Explanation: There is no common prefix among the input strings.
```
Note:

All given inputs are in lowercase letters a-z.

In [1]:
# Solution 1: intuitive solution, check horizontally on all strings
class Solution:
    def longestCommonPrefix(self, strs):
        """
        :type strs: List[str]
        :rtype: str
        """
        if strs is None: # None case
            return None
        if not strs:  # strs is empty
            return ""
    
        cs = ''
        n = min(len(s) for s in strs)
        for i in range(n):
            k = 0
            for s in strs:
                if s[i] != strs[0][i]:
                    break
                k = k + 1
            if k != len(strs):
                break
            cs = cs + strs[0][i]
        return cs

In [3]:
# Solution 2: Solution, check 
# Time O(s), Space O(1)
class Solution(object):
    def longestCommonPrefix(self, strs):
        if len(strs) == 0: # including None and empty cases
            return ""
        for i in range(len(strs[0])):
            for j in range(1, len(strs)):
                if (i == len(strs[j]) or strs[j][i] != strs[0][i]):
                    return strs[0][:i]  ## return previous checked strings
        return strs[0]

In [7]:
# Solution 3: Horizontal scanning
# Time O(s), Space O(1)
class Solution(object):
    def longestCommonPrefix(self, strs):
        if len(strs) == 0:
            return ""
        prefix = strs[0]
        for i in range(1,len(strs)):
            while strs[i].find(prefix) != 0:
                prefix = prefix[: len(prefix) - 1]
            if not prefix:
                return ""
        return prefix

In [6]:
# Solution 4: Divide and conquer
class Solution(object):
    def longestCommonPrefix(self, strs):
        def longestCommonPrefixC(strs, l, r):
            if l == r:
                return strs[l]
            else:
                mid = (l + r)//2
                lcpLeft = longestCommonPrefixC(strs, l, mid)
                lcpRight = longestCommonPrefixC(strs, mid+1, r)
                return commonPrefix(lcpLeft, lcpRight)
        
        def commonPrefix(left, right):
            minL = min(len(left), len(right))
            for i in range(minL):
                if left[i] != right[i]:
                    return left[:i]
            return left[:minL]
                
        if len(strs) == 0:
            return ""
        return longestCommonPrefixC(strs, 0, len(strs) - 1)

In [8]:
# Solution 5: Binary Search
# Time complexity : O(S*log(n)), where S is the sum of all characters in all strings.
# The algorithm makes log(n) iterations, for each of them there are S = m*n comparisons, which gives in total O(S*log(n)) time complexity.
# Space complexity : O(1). We only used constant extra space. 

class Solution:
    def longestCommonPrefix(self, strs):
        """
        :type strs: List[str]
        :rtype: str
        """
        def isCommonPrefix(strs, l):
            """
            :type strs: List[str]
            :type l: int
            :rtype: bool
            """
            str1 = strs[0][:l]
            for i in range(len(strs)):
                if strs[i].find(str1) != 0:
                    return False
            return True
        
        if len(strs) == 0:
            return ""
        minLen = min(len(s) for s in strs)
        low = 1
        high = minLen
        while low <= high:
            mid = (low + high) // 2
            if isCommonPrefix(strs, mid):
                low = mid + 1
            else:
                high = mid - 1
        return strs[0][:(low+high)//2]

# Solution 6: Using Tire Tree. Build a Tire tree and count the longest common prefix

We could optimize LCP queries by storing the set of keys S in a Trie. For more information about Trie, please see this article [Implement a trie (Prefix trie)](https://leetcode.com/articles/implement-trie-prefix-tree/). 

In a Trie, each node descending from the root represents a common prefix of some keys. But we need to find the longest common prefix of a string q and all key strings. This means that we have to find the deepest path from the root, which satisfies the following conditions: 

- it is prefix of query string q 
- each node along the path must contain only one child element. 

Otherwise the found path will not be a common prefix among all strings. 

* the path doesn't comprise of nodes which are marked as end of key. Otherwise the path couldn't be a prefix a of key which is shorter than itself.

Algorithm

The only question left, is how to find the deepest path in the Trie, that fulfills the requirements above. The most effective way is to build a trie from [S1…Sn] strings. Then find the prefix of query string q in the Trie. We traverse the Trie from the root, till it is impossible to continue the path in the Trie because one of the conditions above is not satisfied.

![Figure 4. Finding the longest common prefix of strings using Trie](./images/14_lcp_trie.png)

In [18]:
# Solution 6: Trie tree method
class TrieNode:
    isEnd = False  # whether this node is an end of a word
    links = dict()  # map a char to the child node    
    def containsKey(self, c):
        """
        :type c: char
        :rtype: bool
        """
        return c in self.links.keys()
    
    def put(self, c, node):
        """
        :type c: char
        :type node: TrieNode
        """
        self.links[c] = node
    
    def get(self, c):
        return self.links[c]


    
class Trie:
    def __init__(self):
        self.root = TrieNode()
    
    def insert(self, word):
        node = self.root
        for c in word:
            if not node.containsKey(c):
                node.put(c, TrieNode())
            node = node.get(c)
        node.isEnd = True
    
    def search(self, word):
        """
        :type word: str
        :rtype: bool
        """
        node = self.searchPrefix(word)
        retrun (node != None and node.isEnd())
        
    def searchLongestPrefix(self, word):
        node = self.root
        prefix = ""
        for c in word:
            if node.containsKey(c) and len(node.links) == 1 and not node.isEnd:
                prefix = prefix + c
                node = node.get(c)
            else:
                return prefix
        return prefix

    
class Solution:
    def longestCommonPrefix(self, strs):
        """
        :type strs: List[str]
        :rtype: str
        """
        if len(strs) == 0:
            return ""
        trie = Trie()
        for i in range(0, len(strs)):
            trie.insert(strs[i])
        return trie.searchLongestPrefix(strs[0])
    